# Day 3: Data Cleaning & Preparation

## Cleaning Principles
- Raw data remains untouched in `data/raw/`
- All transformations are explicit and reproducible
- Cleaning decisions are documented and justified
- Fraud labels (`is_fraud`) are preserved

## Planned Cleaning Rules
1. Parse timestamps to datetime and drop invalid timestamps
2. Resolve duplicated `transaction_id` records
3. Standardise amount fields (numeric coercion)
4. Remove or flag invalid transaction amounts (≤ 0)
5. Normalise categorical text fields
6. Handle missing values using transparent rules

In [1]:
# Core libraries
import pandas as pd
import numpy as np

# Load raw dataset (do NOT modify raw files)
RAW_PATH = "../data/raw/nova_pay_combined.csv"
df = pd.read_csv(RAW_PATH)

# Keep an original copy for reference
df_raw = df.copy()

df.shape

(11400, 26)

In [2]:
# Convert timestamp to datetime
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")

# Count invalid timestamps
invalid_timestamps = df["timestamp"].isna().sum()
invalid_timestamps

61

In [3]:
# Drop rows with invalid timestamps
df = df.dropna(subset=["timestamp"])

df.shape

(11339, 26)

In [4]:
# Identify duplicate transaction IDs
dup_count = df.duplicated(subset=["transaction_id"]).sum()
dup_count

199

In [5]:
# Sort so the latest transaction comes last
df = df.sort_values("timestamp")

# Drop duplicates, keeping the latest record per transaction_id
df = df.drop_duplicates(subset=["transaction_id"], keep="last")

df.shape

(11140, 26)

In [6]:
# Convert source amount to numeric (non-numeric -> NaN)
df["amount_src_num"] = pd.to_numeric(df["amount_src"], errors="coerce")

# amount_usd is already numeric, but ensure consistency
df["amount_usd_num"] = pd.to_numeric(df["amount_usd"], errors="coerce")

# Check how many values failed conversion
df[["amount_src_num", "amount_usd_num"]].isna().sum()

amount_src_num      4
amount_usd_num    300
dtype: int64

In [7]:
# --- Amount cleaning step 2: Identify logically invalid values ---

# A transaction amount should be strictly positive (> 0)
invalid_amount_mask = (
    (df["amount_src_num"] <= 0) |
    (df["amount_usd_num"] <= 0)
)

# Count how many rows violate this rule
invalid_amounts_before = invalid_amount_mask.sum()
invalid_amounts_before

100

In [8]:
# Inspect a sample of transactions with invalid amounts
df.loc[invalid_amount_mask, ["transaction_id", "amount_src_num", "amount_usd_num"]].head(10)

,transaction_id,amount_src_num,amount_usd_num
328,3bcab4c7-dc2e-416c-b57e-266aafe46a47,-37.08,37.08
564,8e0f5d06-a518-4b0c-ab83-5208837bc5be,-176.92,176.92
576,064c0207-1b3c-4328-b0b9-7ea68a70ee1f,-702.71,702.71
602,1d2465a1-14b6-48d9-a74d-76a2dddf5e65,-203.83,150.83
655,57b6e089-e25b-4213-8e75-df2c0ab23432,-82.80,82.80
788,1d2ddf81-e4c7-4684-bbed-c1b6446c729e,-143.13,143.13
798,883403f5-fd2f-4b39-824e-9710485a983e,-168.96,211.20
1311,e9c7f2cb-f2de-4603-b38f-f2f66ec1eee1,-165.36,NaN
1418,ae0c1cd8-7f7e-4424-938f-7f7242ff197f,-504.71,504.71
1429,873a17a6-91b1-434c-9908-2bf57fefa746,-144.10,144.10


In [9]:
# --- Amount cleaning step 3: Impute missing (NaN) values with median ---

# Compute medians from the current (converted) data
src_median = df["amount_src_num"].median()
usd_median = df["amount_usd_num"].median()

src_median, usd_median

(159.05, 163.64499999999998)

In [10]:
# Fill NaN values in the numeric amount columns with the median
df["amount_src_num"] = df["amount_src_num"].fillna(src_median)
df["amount_usd_num"] = df["amount_usd_num"].fillna(usd_median)

# Verify that no NaNs remain in amount columns
df[["amount_src_num", "amount_usd_num"]].isna().sum()

amount_src_num    0
amount_usd_num    0
dtype: int64

In [11]:
# --- Amount cleaning step 4: Confirm invalid amounts remain ---

invalid_amount_mask = (
    (df["amount_src_num"] <= 0) |
    (df["amount_usd_num"] <= 0)
)

invalid_amounts_after = invalid_amount_mask.sum()
invalid_amounts_after

100

In [12]:
# --- Amount cleaning step 5: Drop transactions with invalid amounts ---

# Keep only rows with strictly positive amounts
df = df[
    (df["amount_src_num"] > 0) &
    (df["amount_usd_num"] > 0)
]

df.shape

(11040, 28)

In [13]:
# Save cleaned dataset for downstream analysis
OUTPUT_PATH = "../data/processed/cleaned_transactions.csv"
df.to_csv(OUTPUT_PATH, index=False)

OUTPUT_PATH

'../data/processed/cleaned_transactions.csv'